# Controlling With Noisy Measurements

In this exercise, we'll control the coaxial drone using measurements obtained by an altitude sensor. 
We will follow conventional notation and have $x_t$ represent the true state of the drone at time t, $z_t$ the measurement (of altitude) made at time t, and $\hat{x}_t$ the estimate of the true state at time t.

In [1]:
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

import numpy as np 
import math
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import jdc
from ipywidgets import interactive
from CoaxialDrone import CoaxialCopter
from PIDcontroller import PIDController_with_ff
from PathGeneration import flight_path
from DronewithPIDControllerParameters import DronewithPID

pylab.rcParams['figure.figsize'] = 10, 10

The sensor used by the quadrotor estimates the altitude with a weighted average defined as:

$$
\hat{x}_{t} = \alpha \hat{x}_{t-1} + (1-\alpha)z_t
$$

In [2]:
class Sensor:
    def __init__(self,
                 x_hat,            # State estimate
                 alpha             # Alpha value used in averaging
                ):
        '''
        Initialize the Sensor object with initial altitude estimation and the alpha value for the exponential averaging. 
        '''
        self.x_hat = x_hat         
        self.alpha = alpha         
        
    def measure(self, 
                x_t,               # Drone's true state
                sigma = 0.01       # Gaussian's variance
               ):
        '''
        We simulate a realistic altitude measurement by adding Gaussian noise to the true measurement.
        '''
        self.z_t = x_t + np.random.normal(0.0, sigma)
        return self.z_t


    def estimate(self, z_t):
        '''
        We then estimate the drone's altitude using the weighted average method.
        '''
        self.x_hat = self.alpha * self.x_hat + (1 - self.alpha) * z_t
        return self.x_hat

Let us consider a case when at $t=0$ we will ask drone in change the altitude by 1 meter from the stationary position represented as $z=0$ and $\dot{z}=0$. Thus we will generate the desired path with just $z=-1$ for all time $t$. 

In [3]:
total_time = 10.0   # Total Flight time 
dt = 0.01           # A time interval between measurements 

t, z_path, z_dot_path, z_dot_dot_path =  flight_path(total_time, dt,'constant' )

Let's compare the path executed by the drone when we are using the measured altitude to control the controller instead of the actual value which we assumed that was magically given to us.

In [4]:
FlyingDrone = DronewithPID(z_path, z_dot_path, z_dot_dot_path, t, dt, IMU)

In [5]:
interactive_plot = interactive(FlyingDrone.PID_controler_with_measured_values, 
                               k_p=(5.0, 35.0, 1),
                               k_d=(0.0, 10, 0.5), 
                               k_i=(0.0, 10, 0.5), 
                               mass_err =(0.7, 1.31, 0.01),
                               sigma=(0.0, 0.1, 0.001))
output = interactive_plot.children[-1]
output.layout.height = '800px'
interactive_plot

A Jupyter Widget

In this section, we will use the estimated value of the altitude based on the averaging to control the drone instead of relying only on the last measurement value.

In [6]:
interactive_plot = interactive(FlyingDrone.PID_controler_with_estimated_values, 
                               k_p=(5.0, 35.0, 1),
                               k_d=(0.0, 10, 0.5), 
                               k_i=(0.0, 10, 0.5), 
                               mass_err =(0.7, 1.31, 0.01),
                               sigma = (0.0, 0.1, 0.001),
                               alpha = (0.51, 0.99, 0.01))
output = interactive_plot.children[-1]
output.layout.height = '800px'
interactive_plot

A Jupyter Widget

# Questions:
---
* Is the magnitude of the steady-state error higher or lower when using the weighted average compared to the using the directly measured values?
* Does the drone take shorter or longer time to converge at the desired altitude using the weighted averaging for altitude estimation relative to using the direct altitude measurement?
---